In [1]:
import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifierGeneric
from textgrad.loss import TextLoss

In [2]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro", override=True)

/Users/eugeniusms/Development/SKRIPSI/sevet/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
initial_solution = """
To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:
x = (-b ± √(b^2 - 4ac)) / 2a
a = 3, b = -7, c = 2
x = (7 ± √((-7)^2 + 4(3)(2))) / 6
x = (7 ± √73) / 6
The solutions are:
x1 = (7 + √73)
x2 = (7 - √73)"""

question = Variable("Solve 3x^2-7x+2=0",
                    requires_grad=False,
                    role_description="math question")

instruction = Variable("""You will answering a math question. 
                        Please using step-by-step explanation. Be super concise.""",
                        requires_grad=False,
                        role_description="instruction")

# Calculation Using Steps
calculation = Variable(initial_solution,
                        requires_grad=True,
                        role_description="solution to the math question")


In [4]:

verification_task_prompts = [
    "Verify any miss on calculation, if any misses please revise calculation based on misses.",
    "Imagine you're a senior mathematician, suggest revised calculation."
]

verifier = TextualVerifierGeneric(
    verifier_engine=engine, 
    use_cot_generation=False,
    use_step_breakdown=False,
    verification_task_prompts=verification_task_prompts,
    enable_logging=True
)

verification_prompt = Variable("""Evaluate loss value for correctness and reasonable magnitude.""",
                              requires_grad=False,
                              role_description="system prompt")

verified_result = verifier.verify(instance=question,
                                  instruction=instruction,
                                  calculation=calculation)

print("VERIFIED RESULT:", verified_result.value)

PROMPTS ['Verify any miss on calculation, if any misses please revise calculation based on misses.', "Imagine you're a senior mathematician, suggest revised calculation."]
Generating variant 0 ...
Generating variant 1 ...
GENERATED VARIANTS ['To solve the equation 3x² - 7x + 2 = 0, we use the quadratic formula:\nx = (-b ± √(b² - 4ac)) / 2a\na = 3, b = -7, c = 2\nx = (7 ± √((-7)² - 4(3)(2))) / (2*3)\nx = (7 ± √(49 - 24)) / 6\nx = (7 ± √25) / 6\nx = (7 ± 5) / 6\nThe solutions are:\nx₁ = (7 + 5) / 6 = 2\nx₂ = (7 - 5) / 6 = 1/3\n', 'To solve the equation 3x² - 7x + 2 = 0, we use the quadratic formula:\n\nx = (-b ± √(b² - 4ac)) / 2a\n\nHere, a = 3, b = -7, and c = 2.\n\nx = (7 ± √((-7)² - 4 * 3 * 2)) / (2 * 3)\nx = (7 ± √(49 - 24)) / 6\nx = (7 ± √25) / 6\nx = (7 ± 5) / 6\n\nThe solutions are:\nx₁ = (7 + 5) / 6 = 12/6 = 2\nx₂ = (7 - 5) / 6 = 2/6 = 1/3\n']
CALCULATION 
To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:
x = (-b ± √(b^2 - 4ac)) / 2a
a = 3, b = -7, c = 2
x = 